In [ ]:
# https://www.kaggle.com/code/deepakmys/riiid-eda-with-dask
# https://docs.dask.org/en/stable/dataframe.html

In [1]:
# Import the tables of the data set as dataframes.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.api.types import is_numeric_dtype
import dask.dataframe as dd
import dask

# from dask.distributed import Client

# client = Client("tcp://127.0.0.1:46321")
# client

DATA_DIR = '../mlbd-2023/data/'

# users = pd.read_csv('{}/full_users.csv'.format(DATA_DIR), index_col=0)
# events = pd.read_csv('{}/full_events.csv'.format(DATA_DIR), index_col=0)
# subtasks = pd.read_csv('{}/full_subtasks.csv'.format(DATA_DIR), index_col=0)

users = dd.read_csv('{}/full_users.csv'.format(DATA_DIR))
events = dd.read_csv('{}/full_events.csv'.format(DATA_DIR))
subtasks = dd.read_csv('{}/full_subtasks.csv'.format(DATA_DIR), dtype={'availableNumbers': 'object',
       'interval': 'object',
       'numberRepresentations': 'object',
       'solutionRepresentation': 'object'})


In [3]:
print(f"There are {len(users)} users in the data with {len(users[users['start'].isna() == 1])} nans")


There are 64932 users in the data with 0 nans


In [4]:
users.info(buf=None, verbose=True, memory_usage=True)

<class 'dask.dataframe.core.DataFrame'>
RangeIndex: 64932 entries, 0 to 64931
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   user_id            64932 non-null      int64
 1   learning_time_ms   64932 non-null      int64
 2   logged_in_time_ms  64932 non-null      int64
 3   language           64504 non-null      object
 4   country            48778 non-null      object
 5   start              64932 non-null      object
 6   end                64932 non-null      object
dtypes: object(4), int64(3)
memory usage: 3.5 MB


In [5]:
events.info(buf=None, verbose=True, memory_usage=True)

<class 'dask.dataframe.core.DataFrame'>
Int64Index: 2185200 entries, 0 to 115044
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   event_id          2185200 non-null      int64
 1   user_id           2185200 non-null      int64
 2   mode              2153208 non-null      object
 3   game_name         2153208 non-null      object
 4   learning_time_ms  2153208 non-null      float64
 5   subtasks          2153208 non-null      object
 6   number_range      2153208 non-null      object
 7   start             2185200 non-null      object
 8   end               2164557 non-null      object
 9   skill_id          2153208 non-null      float64
10   type              2185200 non-null      object
dtypes: object(7), float64(2), int64(2)
memory usage: 183.4 MB


In [6]:
subtasks.info(buf=None, verbose=True, memory_usage=True)

<class 'dask.dataframe.core.DataFrame'>
Int64Index: 3502884 entries, 0 to 387588
Data columns (total 48 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   subtask_id                  3502884 non-null      int64
 1   event_id                    3502884 non-null      int64
 2   user_id                     3502884 non-null      int64
 3   aim                         676881 non-null      float64
 4   answer                      3382635 non-null      object
 5   answerMode                  223730 non-null      object
 6   availableNumbers            7494 non-null      object
 7   correct                     3502884 non-null      bool
 8   correctAnswerObject         2971438 non-null      object
 9   correctNumber               89100 non-null      float64
10   destination                 386650 non-null      object
11   distance                    177284 non-null      float64
12   hasProperResult             3502884 no

In [3]:
%time events[['Year', 'Week', 'Day']] = dask.dataframe.to_datetime(events['start']).dt.isocalendar()
events.head()

CPU times: user 80.3 ms, sys: 54.3 ms, total: 135 ms
Wall time: 3.11 s


,event_id,user_id,mode,game_name,learning_time_ms,subtasks,number_range,start,end,skill_id,type,Year,Week,Day
0,1,2,NORMAL,Subitizing,13094.0,"[{'hasProperResult': True, 'range': 'R10', 'co...",R10,2020-08-20T07:13:50.876Z,2020-08-20T07:14:30.108Z,1.0,task,2020,34,4
1,3,2,NORMAL,Conversion,15879.0,"[{'answer': 6, 'range': 'R10', 'correct': True...",R10,2020-08-21T07:02:20.112Z,2020-08-21T07:02:36.221Z,3.0,task,2020,34,5
2,4,2,NORMAL,Landing,6075.0,"[{'range': 'R10', 'correct': True, 'lowerBound...",R10,2020-08-24T07:02:59.855Z,2020-08-24T07:03:07.382Z,18.0,task,2020,35,1
3,5,2,NORMAL,Landing,6910.0,"[{'range': 'R10', 'correct': True, 'lowerBound...",R10,2020-08-26T06:47:21.504Z,2020-08-26T06:47:30.050Z,19.0,task,2020,35,3
4,6,2,NORMAL,Calculator,7507.0,"[{'range': 'R20', 'answerMode': 'RESULT', 'cor...",R20,2020-08-26T07:20:58.766Z,2020-08-26T07:21:06.439Z,54.0,task,2020,35,3


In [ ]:
events['year_week'] = ((events['Year'] - 2015) * 53 + events['Week']).astype(int)
ts = events.reset_index().groupby(['user_id','year_week'])['event_id'].count().reset_index()
ts = ts.rename(columns={'event_id':'num_events'})

In [2]:
events_minimal = events.reset_index()[['user_id','event_id','start','skill_id']]

In [3]:
subtasks_minimal = subtasks.reset_index()[['user_id','event_id','correct']]

In [ ]:
events_and_substasks = events_minimal.merge(subtasks_minimal, how='left', on=['event_id','user_id'], suffixes=('_event', '_subtask'))
events_and_substasks['date'] = pd.to_datetime(events_and_substasks['start'])
events_and_substasks[['Year', 'Week', 'Day']] = events_and_substasks['date'].dt.isocalendar()
events_and_substasks['year_week'] = (events_and_substasks['Year'] - 2015) * 53 + events_and_substasks['Week']


In [ ]:
events_and_substasks['week_sequential'] = events_and_substasks.groupby('user_id')['year_week'].apply(lambda x: x - x.iat[0])

In [ ]:
events_and_subtasks.to_csv("events_and_subtasks_full.csv")